In [51]:
import os
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../../")
from pathlib import Path
import numpy as np
import pandas as pd
from tqdm import tqdm
from matplotlib import pyplot as plt
# from src.dataloaders import PointOnlyDataloader, NoiseDataLoader
from src.utils.graphs import split_df_to_graphs

from sklearn.model_selection import train_test_split

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
dataset_dir = Path('../../dataset')
root_file = "farichsim_pi-pi-_45-360deg_1200.0k_ideal_2020-12-24_rndm.root"
root_path = dataset_dir / root_file

dataset = "dataset_100000_with_noise_2e5.csv"


In [43]:
df = pd.read_csv(dataset_dir / dataset)

In [33]:
df

,entry,subentry,x_c,y_c,z_c,t_c,signal
0,0,0,100,55,201.05,2.956575,True
1,0,1,105,31,201.05,3.164390,True
2,0,2,110,32,201.05,3.150230,True
3,0,3,106,67,201.05,2.841400,True
4,0,4,117,37,201.05,3.094911,True
...,...,...,...,...,...,...,...
83645212,99999,799,108,148,201.05,2.433213,False
83645213,99999,800,162,32,201.05,4.979479,False
83645214,99999,801,25,95,201.05,6.684005,False
83645215,99999,802,47,12,201.05,6.480466,False


In [46]:
%%timeit
entry_df = df[df.entry == 0]


48.8 ms ± 239 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [52]:
split_df_to_graphs(entry_df)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/djkostya/projects/farich-detector/.venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_2735447/1571367414.py", line 1, in <module>
    split_df_to_graphs(entry_df)
  File "/home/djkostya/projects/farich-detector/notebooks/ml/../../src/utils/graphs/creater.py", line 8, in split_df_to_graphs
    graphs = []
  File "/home/djkostya/projects/farich-detector/.venv/lib/python3.10/site-packages/pandas/core/generic.py", line 11797, in sum
    return NDFrame.sum(
  File "/home/djkostya/projects/farich-detector/.venv/lib/python3.10/site-packages/pandas/core/generic.py", line 11501, in sum
    return self._min_count_stat_function(
  File "/home/djkostya/projects/farich-detector/.venv/lib/python3.10/site-packages/pandas/core/generic.py", line 11483, in _min_count_stat_function
    return self._reduce(
  File "/home/djkostya/projects/fa

In [35]:
entry_df = df[df.entry == 0][['x_c','y_c', 't_c', 'signal']]

In [39]:
entry_df[(df.t_c >= 0) & (df.t_c <= 5)].reset_index(drop=True)

/tmp/ipykernel_2735447/4278031266.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  entry_df[(df.t_c >= 0) & (df.t_c <= 5)].reset_index(drop=True)


,x_c,y_c,t_c,signal
0,100,55,2.956575,True
1,105,31,3.164390,True
2,110,32,3.150230,True
3,106,67,2.841400,True
4,117,37,3.094911,True
...,...,...,...,...
561,47,170,2.521595,False
562,1,167,0.650147,False
563,83,96,4.902512,False
564,24,153,2.674867,False


In [7]:
entry_df['join']=0
entry_df['index'] = entry_df.index



In [8]:
entry_df

,x_c,y_c,t_c,signal,join,index
0,100,55,2.956575,True,0,0
1,105,31,3.164390,True,0,1
2,110,32,3.150230,True,0,2
3,106,67,2.841400,True,0,3
4,117,37,3.094911,True,0,4
...,...,...,...,...,...,...
794,131,202,6.078539,False,0,794
795,168,101,5.509062,False,0,795
796,83,96,4.902512,False,0,796
797,24,153,2.674867,False,0,797


In [9]:
joined_df = entry_df.merge(entry_df, on='join', how='outer')

In [10]:
joined_df['distance'] = joined_df.apply(lambda x: np.sqrt((x.x_c_x - x.x_c_y)**2 + (x.y_c_x - x.y_c_y)**2 + (x.t_c_x - x.t_c_y)**2), axis=1)

In [14]:
joined_df_with_distance = joined_df[(joined_df.distance > 0) & (joined_df.t_c_x < 5) & (joined_df.t_c_y < 5) & (joined_df.distance < 36)]

In [15]:
joined_df_with_distance


,x_c_x,y_c_x,t_c_x,signal_x,join,index_x,x_c_y,y_c_y,t_c_y,signal_y,index_y,distance
1,100,55,2.956575,True,0,0,105,31,3.164390,True,1,24.516182
2,100,55,2.956575,True,0,0,110,32,3.150230,True,2,25.080620
3,100,55,2.956575,True,0,0,106,67,2.841400,True,3,13.416902
4,100,55,2.956575,True,0,0,117,37,3.094911,True,4,24.759223
5,100,55,2.956575,True,0,0,117,33,3.132797,True,5,27.803436
...,...,...,...,...,...,...,...,...,...,...,...,...
638247,86,29,4.297562,False,0,798,114,27,3.247158,False,645,28.090983
638262,86,29,4.297562,False,0,798,111,52,4.135098,False,660,33.970964
638299,86,29,4.297562,False,0,798,114,10,0.393297,False,697,34.062344
638312,86,29,4.297562,False,0,798,100,23,2.433674,False,710,15.345165


In [13]:
joined_df[joined_df.signal_x & joined_df.signal_y].describe()

,x_c_x,y_c_x,t_c_x,join,index_x,x_c_y,y_c_y,t_c_y,index_y,distance
count,324.000000,324.000000,324.000000,324.0,324.000000,324.000000,324.000000,324.000000,324.000000,324.000000
mean,124.500000,73.555556,3.021098,0.0,8.500000,124.500000,73.555556,3.021098,8.500000,52.747142
std,18.621619,49.155468,0.262489,0.0,5.196152,18.621619,49.155468,0.262489,5.196152,52.300215
min,100.000000,31.000000,2.773486,0.0,0.000000,100.000000,31.000000,2.773486,0.000000,0.000000
25%,114.000000,37.000000,2.828560,0.0,4.000000,114.000000,37.000000,2.828560,4.000000,17.720127
50%,120.500000,63.500000,2.963475,0.0,8.500000,120.500000,63.500000,2.963475,8.500000,33.778996
75%,128.000000,74.000000,3.132797,0.0,13.000000,128.000000,74.000000,3.132797,13.000000,69.771088
max,181.000000,231.000000,3.896714,0.0,17.000000,181.000000,231.000000,3.896714,17.000000,203.962095


In [16]:
edjes = pd.DataFrame([joined_df_with_distance.index_x, joined_df_with_distance.index_y])

In [18]:
edjes_array = edjes.to_numpy().T

In [21]:
edjes_array.shape

(19618, 2)

In [22]:
nodes = entry_df[entry_df.t_c<5][['x_c', 'y_c', 't_c']]

In [24]:
x = nodes.to_numpy()

In [25]:
from torch_geometric.data import Data

/home/djkostya/projects/farich-detector/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [27]:
graph = Data(x=x, edge_index=edjes)

In [6]:
entry_df[entry_df.t_c<=5]

,entry,subentry,x_c,y_c,z_c,t_c,signal
0,0,0,100,55,201.05,2.956575,True
1,0,1,105,31,201.05,3.164390,True
2,0,2,110,32,201.05,3.150230,True
3,0,3,106,67,201.05,2.841400,True
4,0,4,117,37,201.05,3.094911,True
...,...,...,...,...,...,...,...
791,0,791,47,170,201.05,2.521595,False
793,0,793,1,167,201.05,0.650147,False
796,0,796,83,96,201.05,4.902512,False
797,0,797,24,153,201.05,2.674867,False


In [85]:
df

,entry,subentry,x_c,y_c,z_c,t_c,signal
0,0,0,100,55,201.05,2.956575,True
1,0,1,105,31,201.05,3.164390,True
2,0,2,110,32,201.05,3.150230,True
3,0,3,106,67,201.05,2.841400,True
4,0,4,117,37,201.05,3.094911,True
...,...,...,...,...,...,...,...
83645212,99999,799,108,148,201.05,2.433213,False
83645213,99999,800,162,32,201.05,4.979479,False
83645214,99999,801,25,95,201.05,6.684005,False
83645215,99999,802,47,12,201.05,6.480466,False


In [5]:
entities = df.entry.unique()

In [6]:
train, test = train_test_split(entities, test_size=0.4, random_state=42, shuffle=True)

In [7]:
df.t_c = df.t_c.round(2)

In [33]:
import time

In [35]:
start = time.time()
get_matrix_from_entry(df, 1)
print(time.time() - start)

0.11068010330200195


In [38]:
result = []

matrixes_dir = dataset_dir / 'matrixes'

matrixes_dir.mkdir(exist_ok=True, parents=True)
matrixes_dir = matrixes_dir.absolute().resolve()

for train_id, i in tqdm(enumerate(train), total=len(train)):
    matrixes = get_matrix_from_entry(df, i)
    for matrixe_idx, m in enumerate(matrixes):
        filename = f'{train_id}_{matrixe_idx}_matrix.npy'
        with open(matrixes_dir/filename, 'wb') as f:
            np.save(f, m[0])
        result.append((str((matrixes_dir/filename)), m[1]))


  0%|          | 8/60000 [00:10<22:23:59,  1.34s/it]


KeyboardInterrupt: 

In [27]:
matrixes = get_matrix_from_entry(df, 1)

(array([[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],
 
        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],
 
        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],
 
        ...,
 
        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],


In [ ]:
result

In [17]:
matrix_df = pd.DataFrame(result, columns=['matrix_pth', 'ratio'])

In [19]:
matrix_df.matrix_pth = matrix_df.matrix_pth.apply(lambda x: Path(x).resolve().__str__())

In [20]:
matrix_df.to_csv(dataset_dir/"matrix.csv", index=False)

In [162]:
train_df = pd.concat(m)


  0%|          | 33/60000 [00:02<1:01:25, 16.27it/s]/var/folders/2h/pvzr32ss6hd_92k0r59f4k940000gn/T/ipykernel_72088/367963849.py:14: RuntimeWarning: invalid value encountered in scalar divide
  matrixes.append((matrix, current_signal/total_signal))

  0%|          | 47/60000 [00:03<1:00:36, 16.49it/s]/var/folders/2h/pvzr32ss6hd_92k0r59f4k940000gn/T/ipykernel_72088/367963849.py:14: RuntimeWarning: invalid value encountered in scalar divide
  matrixes.append((matrix, current_signal/total_signal))

  0%|          | 107/60000 [00:07<1:05:18, 15.29it/s]/var/folders/2h/pvzr32ss6hd_92k0r59f4k940000gn/T/ipykernel_72088/367963849.py:14: RuntimeWarning: invalid value encountered in scalar divide
  matrixes.append((matrix, current_signal/total_signal))

  0%|          | 201/60000 [00:13<1:03:19, 15.74it/s]/var/folders/2h/pvzr32ss6hd_92k0r59f4k940000gn/T/ipykernel_72088/367963849.py:14: RuntimeWarning: invalid value encountered in scalar divide
  matrixes.append((matrix, current_signal/total_sig

KeyboardInterrupt: 

In [25]:
def get_matrix_from_entry(df, entry):
    entry_df = df[df.entry == entry]

    return get_matrix_from_df(entry_df)

In [26]:
def get_matrix_from_df(entry_df, window_size=500):
    matrixes = []
    total_signal = entry_df.signal.sum()
    if total_signal == 0:
        return matrixes
    times = np.arange(0, 13, 0.01)
    windows = rolling_window(times, window_size, 200)
    for idxs in windows:
        idx2val = {val: idx for idx, val in enumerate(idxs)}
        matrix = np.zeros((window_size, 240, 240))
        entries_for_idx = entry_df[entry_df.t_c.isin(idxs)]
        current_signal = entries_for_idx.signal.sum()
        for idx, row in entries_for_idx.iterrows():
            matrix[idx2val[row['t_c']], row['x_c']-1, row['y_c']-1] = 1

        matrixes.append((matrix, current_signal/total_signal))
    return matrixes


In [152]:
first_entry = df[df.entry == 10]
first_entry.t_c = first_entry.t_c.round(2)
get_matrix_from_df(first_entry)

/var/folders/2h/pvzr32ss6hd_92k0r59f4k940000gn/T/ipykernel_72088/1409683054.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_entry.t_c = first_entry.t_c.round(2)


,Matrix,target_acc
0,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",0.000000
1,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",0.859649
2,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",0.859649
3,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",0.017544


/var/folders/2h/pvzr32ss6hd_92k0r59f4k940000gn/T/ipykernel_72088/459580876.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_entry.t_c = first_entry.t_c.round(2)


In [90]:
first_entry.t_c.max()

7.0

In [91]:
times = np.arange(0, 10, 0.01)

In [92]:
times

array([0.  , 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ,
       0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21,
       0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32,
       0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43,
       0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54,
       0.55, 0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65,
       0.66, 0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76,
       0.77, 0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87,
       0.88, 0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98,
       0.99, 1.  , 1.01, 1.02, 1.03, 1.04, 1.05, 1.06, 1.07, 1.08, 1.09,
       1.1 , 1.11, 1.12, 1.13, 1.14, 1.15, 1.16, 1.17, 1.18, 1.19, 1.2 ,
       1.21, 1.22, 1.23, 1.24, 1.25, 1.26, 1.27, 1.28, 1.29, 1.3 , 1.31,
       1.32, 1.33, 1.34, 1.35, 1.36, 1.37, 1.38, 1.39, 1.4 , 1.41, 1.42,
       1.43, 1.44, 1.45, 1.46, 1.47, 1.48, 1.49, 1.

In [93]:
times.strides
window_size = 500

In [94]:
from numpy.lib.stride_tricks import sliding_window_view

x = sliding_window_view(times, window_size)

In [95]:
x

array([[0.  , 0.01, 0.02, ..., 4.97, 4.98, 4.99],
       [0.01, 0.02, 0.03, ..., 4.98, 4.99, 5.  ],
       [0.02, 0.03, 0.04, ..., 4.99, 5.  , 5.01],
       ...,
       [4.98, 4.99, 5.  , ..., 9.95, 9.96, 9.97],
       [4.99, 5.  , 5.01, ..., 9.96, 9.97, 9.98],
       [5.  , 5.01, 5.02, ..., 9.97, 9.98, 9.99]])

In [10]:
def rolling_window(a, window, step):
    shape = a.shape[:-1] + ((a.shape[-1] - window + 1)//step, window)
    strides = (a.strides[0] * step,) + (a.strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)

In [97]:
windows = rolling_window(times, 400, 50)

In [98]:
total_signal = first_entry.signal.sum()

In [115]:
for idxs in windows:
    idx2val = {val: idx for idx, val in enumerate(idxs)}
    matrix = np.zeros((400, 240, 240))
    entries_for_idx = first_entry[first_entry.t_c.isin(idxs)]
    current = entries_for_idx.signal.sum()
    for idx, row in entries_for_idx.iterrows():
        matrix[idx2val[row['t_c']], row['x_c']-1, row['y_c']-1] = 1


0.9444444444444444


In [116]:
idx2val[2.96]

296

In [121]:
matrix[296].shape

(240, 240)

In [118]:
np.where(matrix == 1)

(array([  0,   2,   2,   2,   3,   3,   3,   3,   4,   4,   8,   9,  11,
         11,  12,  12,  14,  15,  16,  18,  18,  18,  19,  19,  21,  23,
         24,  25,  26,  29,  30,  30,  31,  32,  34,  36,  38,  38,  39,
         39,  39,  39,  42,  42,  43,  46,  49,  49,  49,  50,  50,  52,
         55,  55,  58,  58,  62,  63,  64,  65,  65,  66,  67,  68,  71,
         72,  73,  73,  73,  74,  75,  76,  76,  77,  78,  78,  86,  86,
         86,  87,  88,  88,  88,  89,  89,  89,  92,  93,  93,  97,  98,
         98,  98, 100, 100, 101, 101, 102, 102, 104, 104, 105, 106, 106,
        106, 108, 110, 111, 111, 116, 116, 117, 117, 118, 119, 120, 120,
        120, 121, 122, 122, 123, 124, 125, 125, 125, 126, 126, 127, 128,
        132, 135, 135, 136, 136, 136, 141, 141, 142, 145, 145, 145, 146,
        146, 147, 147, 147, 150, 150, 150, 150, 150, 151, 152, 153, 154,
        155, 155, 155, 155, 157, 159, 159, 159, 159, 160, 160, 162, 162,
        167, 168, 169, 171, 172, 172, 172, 174, 176

In [104]:
first_entry

,entry,subentry,x_c,y_c,z_c,t_c,signal
0,0,0,100,55,201.05,2.96,True
1,0,1,105,31,201.05,3.16,True
2,0,2,110,32,201.05,3.15,True
3,0,3,106,67,201.05,2.84,True
4,0,4,117,37,201.05,3.09,True
...,...,...,...,...,...,...,...
794,0,794,131,202,201.05,6.08,False
795,0,795,168,101,201.05,5.51,False
796,0,796,83,96,201.05,4.90,False
797,0,797,24,153,201.05,2.67,False
